In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_csv("ames.csv")
df.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YearSold,SaleType,SaleCondition,SalePrice
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141.0,31770.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,0.0,No_Pool,No_Fence,None,0.0,5,2010,WD,Normal,215000
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80.0,11622.0,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,0.0,No_Pool,Minimum_Privacy,None,0.0,6,2010,WD,Normal,105000
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81.0,14267.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,0.0,No_Pool,No_Fence,Gar2,12500.0,6,2010,WD,Normal,172000
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93.0,11160.0,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,0.0,No_Pool,No_Fence,None,0.0,4,2010,WD,Normal,244000
4,Two_Story_1946_and_Newer,Residential_Low_Density,74.0,13830.0,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,0.0,No_Pool,Minimum_Privacy,None,0.0,3,2010,WD,Normal,189900


In [ ]:
!pip install Pyfhel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for Pyfhel: filename=Pyfhel-3.4.2-cp310-cp310-linux_x86_64.whl size=8501319 sha256=33b2f6442b92e1059086d22821240c31d0b5e1d1dc104e89f3b94c296510d5e8
  Stored in directory: /root/.cache/pip/wheels/74/7b/34/876b6d79c02f65e68e4e6f262735c2347cf77581ac50ca470d
Successfully built Pyfhel


In [ ]:
data=pd.DataFrame()
data["x"]=df["TotalBsmtSF"]
data["y"]=df["GrLivArea"]
data.head()

,x,y
0,1080.0,1656.0
1,882.0,896.0
2,1329.0,1329.0
3,2110.0,2110.0
4,928.0,1629.0


In [ ]:
import math
import numpy as np
from Pyfhel import Pyfhel, PyCtxt
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

def distance(d1,d2):
    return math.sqrt(math.pow(float(d1),2)+ math.pow(float(d2),2))

def l2_dist_sq_ctxt(vec_1: PyCtxt, vec_2: PyCtxt, HE: Pyfhel):
    """
    Calculates the encrypted squared l2 norm between encrypted vectors vec_1 and vec_2
    """
    dist_sq = ~(vec_1 - vec_2)**2
    HE.relinKeyGen()  # relinearize after ctxt-ctxt mults
    #dist_sq = HE.cumul_add(dist_sq)
    return dist_sq


m = 100
P = [Point(0, 0) for _ in range(m)]

for i in range(m):
    P[i].x, P[i].y = data.loc[i, 'x'], data.loc[i, 'y']

k = int(input("Enter the number of clusters: "))
rp_indices = np.random.choice(m, size=k, replace=False)  # Randomly select k indices from the dataset

rp = [Point(0, 0) for _ in range(k)]

for i in range(k):
    rp[i].x, rp[i].y = data.loc[rp_indices[i], 'x'], data.loc[rp_indices[i], 'y']

rx=[]
ry=[]
for i in range(k):
    rx.append(rp[i].x)
    ry.append(rp[i].y)
px=[]
py=[]
for i in range(m):
    px.append(P[i].x)
    py.append(P[i].y)

HE = Pyfhel()
ckks_params = {'scheme': 'CKKS', 'n': 8192, 'scale': 2**40, 'qi_sizes': [60, 40, 40, 60]}
HE.contextGen(**ckks_params)
HE.keyGen()
HE.rotateKeyGen()

crp = [Point(0,0) for _ in range(k)]
cp = [Point(0,0) for i in range(m)]

for j in range(0,m):
    cp[j].x=HE.encrypt(px[j:j+HE.get_nSlots()])
    cp[j].y=HE.encrypt(py[j:j+HE.get_nSlots()])

while True:
    for j in range(0,k):
        crp[j].x=HE.encrypt(rx[j:j+HE.get_nSlots()])
        crp[j].y=HE.encrypt(ry[j:j+HE.get_nSlots()])
    arr = [[] for _ in range(k)]
    mp = Point(0,0)
    count = [0] * k

    for i in range(m):
        d=[]
        for j in range(k):
            d1 = HE.decrypt(l2_dist_sq_ctxt(cp[i].x, crp[j].x, HE))
            d2 = HE.decrypt(l2_dist_sq_ctxt(cp[i].y, crp[j].y, HE))
            d.append(distance(d1[0],d2[0]))
        c = d.index(min(d))
        arr[c].append(P[i])
        count[c] += 1



    res = [Point(0,0) for _ in range(k)]

    for i in range(k):
            sum_x = sum_y = 0.0
            for j in range(count[i]):
                sum_x += arr[i][j].x
                sum_y += arr[i][j].y
            res[i].x=sum_x / count[i]
            res[i].y=sum_y / count[i]

    flag = 0
    for i in range(k):
        for p in range(k):
            if rp[i].x == res[p].x and rp[i].y == res[p].y:
                flag += 1

    if flag == k:
        print("Successssssss....")
        for i in range(k):
            print("Cluster no.:", i + 1)
            for j in range(count[i]):
                print(f"({arr[i][j].x},{arr[i][j].y})", end=" ")
            print()
        break
    else:
        print("Sorry...next trial...")
        for i in range(k):
            rp[i].x = res[i].x
            rp[i].y = res[i].y
            rx[i]=rp[i].x
            ry[i]=rp[i].y

Enter the number of clusters: 7


<ipython-input-31-bf8145a01d33>:16: RuntimeWarning: <Pyfhel Warning> relin_key empty, generating it for relinearization.
  dist_sq = ~(vec_1 - vec_2)**2


Sorry...next trial...
Sorry...next trial...
Sorry...next trial...
Sorry...next trial...
Sorry...next trial...
Sorry...next trial...
Successssssss....
Cluster no.: 1
(1080.0,1656.0) (928.0,1629.0) (926.0,1604.0) (994.0,1804.0) (763.0,1655.0) (789.0,1465.0) (814.0,1674.0) (855.0,1456.0) (855.0,1456.0) (764.0,1626.0) (847.0,1733.0) (756.0,1547.0) (831.0,1659.0) (996.0,1744.0) (650.0,1564.0) (840.0,1680.0) 
Cluster no.: 2
(559.0,1752.0) (384.0,1374.0) (384.0,1430.0) (0.0,1902.0) 
Cluster no.: 3
(2110.0,2110.0) (1650.0,3279.0) (1542.0,2073.0) (1590.0,2334.0) (2330.0,2364.0) (2846.0,2696.0) (1671.0,2250.0) (1393.0,2599.0) (1629.0,2475.0) (1463.0,2622.0) (2033.0,3238.0) 
Cluster no.: 4
(882.0,896.0) (1168.0,1187.0) (864.0,864.0) (1053.0,1173.0) (1004.0,1004.0) (1078.0,1078.0) (1056.0,1056.0) (882.0,882.0) (864.0,864.0) (1069.0,1069.0) (836.0,836.0) (1145.0,1145.0) (1218.0,1218.0) (923.0,923.0) (918.0,918.0) (744.0,752.0) (1040.0,1097.0) (816.0,1012.0) (1109.0,1155.0) (894.0,894.0) (882.0,900.